# Análise Exploratória dos Dados
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
pd.set_option("display.max_columns", 100)

BASE_DIR = Path("data_ons")

df_carga_v = pd.read_parquet(BASE_DIR / "carga_verificada_SECO.parquet")
df_carga_p = pd.read_parquet(BASE_DIR / "carga_programada_SECO.parquet")
df_balanco = pd.read_parquet(BASE_DIR / "balanco_subsistemas.parquet")
df_cmo = pd.read_parquet(BASE_DIR / "cmo_semi_horario_SECO.parquet")
df_gtm = pd.read_parquet(BASE_DIR / "geracao_termica_motivo.parquet")

df_carga_v.head()


In [ ]:
def fix_datetime(df, col):
    df[col] = pd.to_datetime(df[col], utc=True, errors="coerce")
    df = df.dropna(subset=[col])
    df = df.sort_values(col)
    return df

df_carga_v = fix_datetime(df_carga_v, "din_referenciautc")
df_carga_p = fix_datetime(df_carga_p, "din_referenciautc")
df_balanco = fix_datetime(df_balanco, "din_instante")
df_cmo = fix_datetime(df_cmo, "din_instante") if "din_instante" in df_cmo.columns else fix_datetime(df_cmo, "din_referenciautc")
df_gtm = fix_datetime(df_gtm, "din_instante")

df_balanco.head()